The problem with the code is caused by batched addition

In [2]:
import numpy as np
from tensor import Tensor
from nn import Linear
from tqdm import trange
from datasets import load_dataset

mnist = load_dataset('mnist')

def convert_to_np(example):
    example['np_image'] = np.array(example['image'])
    return example
mnist = mnist.map(convert_to_np)
# this next line is terribly slow for some reason
X_train, Y_train, X_test, Y_test = np.array(mnist['train']['np_image']), np.array(mnist['train']['label']), np.array(mnist['test']['np_image']), np.array(mnist['test']['label'])
  
def layer_init(m, h):
  w = np.random.uniform(-1., 1., size=(m,h))/np.sqrt(m*h)
  return w.astype(np.float32)

def bias_init(h):
  return np.zeros((h,), dtype=np.float32)

lr = 0.01
BS = 64

l1 = Linear(28*28, 128)
l2 = Linear(128, 10)

In [ ]:
losses, accuracies = [], []
for i in (t := trange(1000)):
  samp = np.random.randint(0, X_train.shape[0], size=(BS))  
  x = Tensor(X_train[samp].reshape((-1, 28*28)))
  Y = Y_train[samp]
  y = np.zeros((len(samp),10), np.float32)
  y[range(y.shape[0]),Y] = -1.0
  y = Tensor(y)
  x = l1(x)
  x = x.relu()
  x = x_l2 = l2(x)
  x = x.logsoftmax()
  x = x.mul(y)
  x = x.mean()
  x.backward()
  
  loss = x.data

  cat = np.argmax(x_l2.data, axis=1)
  accuracy = (cat == Y).mean()
  
  # SGD
  l1.step(lr)
  l2.step(lr)
  accuracies.append(accuracy)
  t.set_description(f"loss {loss} accuracy {accuracy}")

loss [0.00696136] accuracy 0.984375: 100%|██████████| 1000/1000 [00:05<00:00, 179.36it/s]


In [ ]:
# Use the test set to evaluate the model


In [1]:
import nn
import tensor as t
import numpy as np

model = nn.Sequential(
    nn.Linear(784, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax()
)


Sequential(<class 'nn.Linear'>, <class 'nn.ReLU'>, <class 'nn.Linear'>, <class 'nn.LogSoftmax'>)


In [ ]:
from tensor import Tensor, Dot
import numpy as np
a = Tensor(np.random.randn(3,4))
b = Tensor(np.random.randn(4,5))


TypeError: Dot.forward() missing 1 required positional argument: 'weight'

In [ ]:
# numpy forward pass
def forward(x):
  x = l1(x)
  x.data = np.maximum(x.data, 0)
  x = l2(x)
  return x.data


def numpy_eval():
  Y_test_preds_out = forward(X_test.reshape((-1, 28*28)))
  Y_test_preds = np.argmax(Y_test_preds_out, axis=1)
  return (Y_test == Y_test_preds).mean()

print(f"test set accuracy is {numpy_eval()}")
